In [1]:
from alchzh_aoc_helpers.nb import AOCWidget
from alchzh_aoc_helpers.aliases import *
A = AOCWidget(2023, 5); A

Output()

In [36]:
line1, *rest = str(A).split("\n\n")
seeds = [int(s) for s in line1.split()[1:]]
ranges = [(seeds[s], seeds[s] + seeds[s+1]) for s in range(0, len(seeds), 2)]

maps = [
    sorted((
        [int(i) for i in l.split()]
        for l in m.split("\n")[1:]
    ), key=lambda x: x[1])
    for m in rest
]

In [44]:
def part1(seed, maps=maps):
    ret = seed
    for map in maps:
        for dest, source, length in map:
            if source <= ret < source + length:
                ret = dest + ret - source
                break
    return ret

print(min(part1(s) for s in seeds))
%timeit min(part1(s) for s in seeds)

227653707
176 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [50]:
from heapq import heappush, heappop

def part2(ranges, maps=maps):
    v = collections.deque(ranges)
    w = []
    for map in maps:
        while v:
            a, b = v.popleft()
            for j, i, k in map:
                off = j - i
                l = i + k                
                if i <= a < l:
                    if l < b:
                        v.append((l, b))
                        heappush(w, (a + off, l + off))
                    else:
                        heappush(w, (a + off, b + off))
                    break
                elif i < b <= l:
                    v.append((a, i))
                    heappush(w, (i + off, b + off))
                    break
                elif a < i and l < b:
                    v.append((a, i))
                    v.append((l, b))
                    heappush(w, (i + off, l + off))
                    break
            else:
                heappush(w, (a, b))
        last_a, last_b = heappop(w)
        while w:
            a, b = heappop(w)
            if a > last_b:
                v.append((last_a, last_b))
                last_a = a
                last_b = b
            else:
                last_b = b
        v.append((last_a, last_b))
    return min(v)[0]

print("Part 2:", part2(ranges))
%timeit rmap(ranges)

Part 2: 78775051
1.66 ms ± 53.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
